In [1]:
class Transaction:
    def __init__(self, plate, lane, lprin_time):
        self.plate = plate
        self.lane = lane
        self.lprin_time = lprin_time
        self.biometry_time = None
        self.appointment_time = None
        self.message_time = None
        self.arm_time = None
        self.print_time = None
        self.exit_time = None
        self.appointment_nbr = None
        self.transaction_id = None

    def to_dict(self):
        return {
            "plate": self.plate,
            "transaction_id": self.transaction_id,
            "appointment_nbr": self.appointment_nbr,
            "lane": self.lane,
            "lprin_time": self.lprin_time,
            "biometry_time": self.biometry_time,
            "appointment_time": self.appointment_time,
            "message_time": self.message_time,
            "arm_time": self.arm_time,
            "print_time": self.print_time,
            "exit_time": self.exit_time
        }

In [3]:
import os

db = []

count = 0

items = {}

last_row = None

SOURCE_PATH = "source"
LOG_FILE_NAME = "log.log"

FULL_FILE_PATH = os.path.join(SOURCE_PATH, LOG_FILE_NAME)

OUTPUT_PATH = "outputs"

def get_lane_and_time(line):
    lane = "Lane" + line.split("Lane")[1].split(" ")[0].replace(")", "").replace("]", "")
    time = line.split('[')[0].strip()
    return lane, time

with open(FULL_FILE_PATH, 'r') as file:
    for line in file:
        count += 1

        data = None

        if "Lane" in line:

            if "KioskLPRMessage received from" in line:
                data = { 'lane': line.split('from:')[1].strip().split(' ')[0].replace('.', ''), 'time': line.split('[')[0].strip(), 'plate': line.split('LPR:')[1].strip() }

                if data['lane'] in items: del items[data['lane']]
                items[data['lane']] = Transaction(data['plate'], data['lane'], data['time'])

            elif "Transaction created ID=" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time, 'id': line.split('ID=')[1].strip() }

                if data['lane'] in items: 
                    items[data['lane']].biometry_time = data['time']
                    items[data['lane']].transaction_id = data['id']
            
            elif "FingerprintDataReadMessage" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }

                if data['lane'] in items: items[data['lane']].biometry_time = data['time']

            elif "Appointment Query interface" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }
                
                if data['lane'] in items: 
                    items[data['lane']].appointment_time = data['time']

                    if "appointments:" in line:
                        appointment_nbr = line.split('appointments:')[1].strip()
                        items[data['lane']].appointment_nbr = appointment_nbr
            
            elif "Message added: SetPedestalDisplay" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }
                
                if data['lane'] in items and "Obrigado" in last_row: 
                    items[data['lane']].message_time = data['time']
            
            elif "Autogate Successful" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }
            
                if data['lane'] in items: items[data['lane']].arm_time = data['time']
            
            elif "Print Job Completed" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }
            
                if data['lane'] in items: items[data['lane']].print_time = data['time']
            
            elif "Closing Transaction because truck has departed the kiosk" in line:
                lane, time = get_lane_and_time(line)
                data = { 'lane': lane, 'time': time }
            
                if data['lane'] in items: 
                    items[data['lane']].exit_time = data['time']
                    db.append(items[data['lane']].to_dict())
                    
                    del items[data['lane']]
                    count += 1

        last_row = line

In [4]:
print(len(db))

2196


In [ ]:
import os
import pandas as pd
from datetime import datetime

OUTPUT_PATH = "outputs"

os.makedirs(OUTPUT_PATH, exist_ok=True)

df = pd.DataFrame(db)

now_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

full_output_file_path = os.path.join(OUTPUT_PATH, f"transactions_{now_str}.xlsx")

df.to_excel(full_output_file_path, index=False)
